# Non - Exhaustive algorithm: Genetic algorithm

## Author: Kristina Kulivnyk

This notebook features the implementation of Genetic algorithm.

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances
from scipy.spatial import distance_matrix

import sys
sys.path.insert(0, '../..')

import random
import time
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import graph_class as gc

%matplotlib inline

GA highly depends on parameters selection. <br>
There are several main types of parameters that need to be assigned: <br>
1. Population -> number of individuals in population
2. Generation -> number of generations (basically how many times we run GA)
3. Mutation rate -> percentage of the population that will mutate
4. Tournament -> number of individuals from population that will tale part in pouranment breeding
5. SelectionTypeParent -> type of selection method for parents. 1 - roulette selection, 2 - tournament selection.
6. SelectionTypePool -> type of selection method for matting pool. 1 - roulette selection, 2 - tournament selection.
7. Elitism -> if TRUE, than we save best obtained "genes" for next generations. If FALSE - not.
8. Timed -> if TRUE, stop algo after 5 min(300 sec).

In [ ]:
def GA_approach(graph, PopulationIntended=10000, GenerationIntended=25000, mutationRateIntended=0.05, tournamentSelectionSizeIntended=350,
                selectionTypeParentIntended = 1, selectionTypePoolIntended = 1, elitismIntended=False, timed=False ):
    
    mutationRate = mutationRateIntended

    tournamentSelectionSize = tournamentSelectionSizeIntended

    elitism = elitismIntended

    PopulationSize = PopulationIntended
    GenerationSize = GenerationIntended
    
    selectionTypeParent=selectionTypeParentIntended
    selectionTypePool = selectionTypePoolIntended

    Cities = []

    Matrix = graph

    class City:
        def __init__(self, id=0):
            self.id = id

        def distanceTo(self, city):
            return Matrix[self.id][city.id]

        def __repr__(self):
            return str("( id = " + str(self.id) + ")")

    class Route:
        def __init__(self, route=None):
            self.route = []
            self.fitness = 0.0
            self.distance = 0
            if route is not None:
                self.route = route
            else:
                for i in range(0, len(Cities)):
                    self.route.append(None)

        def __len__(self):
            return len(self.route)

        def __getitem__(self, index):
            return self.route[index]

        def __setitem__(self, key, value):
            self.route[key] = value

        def __repr__(self):
            geneString = ""
            for i in range(0, len(self.route)):
                geneString += str(self[i]) + "\n "
            return geneString

        def generateIndividual(self):
            for cityIndex in range(0, len(Cities)):
                self.addCity(cityIndex, Cities[cityIndex])
            random.shuffle(self.route)

        def addCity(self, id, city):
            self.route[id] = city
            self.fitness = 0.0
            self.distance = 0

        def getFitness(self):
            if self.fitness == 0:
                self.fitness = 1 / float(self.getDistance())
            return self.fitness

        def getDistance(self):
            if self.distance == 0:
                distance = 0
                len_route = len(self.route)
                B_final = self.route[len_route - 1]

                for i in range(0, len_route - 1):
                    A = self.route[i]
                    B = self.route[i + 1]
                    distance += A.distanceTo(B)

                distance += B_final.distanceTo(self.route[0])  # From Last point(B) To First One
                self.distance = distance
            return self.distance

    class Population:
        def __init__(self, populationSize, initialise):
            self.routes = []
            for i in range(0, populationSize):
                self.routes.append(None)

            if initialise:
                for i in range(0, populationSize):
                    newTour = Route()
                    newTour.generateIndividual()
                    self.routes[i] = newTour

        def __setitem__(self, key, value):
            self.routes[key] = value

        def __getitem__(self, index):
            return self.routes[index]

        def __len__(self):
            return len(self.routes)

        def getFittest(self):
            fittest = self.routes[0]
            for i in range(0, self.populationSize()):
                if fittest.getFitness() <= self[i].getFitness():
                    fittest = self[i]
            return fittest

        def populationSize(self):
            return len(self.routes)

        def getRouletteList(self): # here in the end of list - sum of all fitnesses (max value)
            roiletlist = []
            a = 0.0
            roiletlist.append(a)
            for i in range(0, self.populationSize()):
                a += self[i].getFitness()
                roiletlist.append(a)

            return roiletlist

    def evolvePopulation(population):
        nextPopulation = Population(len(population), False)
        elitismOffset = 0
        if elitism:
            nextPopulation[0] = population.getFittest()
            elitismOffset = 1

        mattingPopulation = mattingCreatorPool(population)
        # Current work
        for i in range(elitismOffset, nextPopulation.populationSize()):
            if (selectionTypeParent== 1) :
                parent1 = rouletteSelection(mattingPopulation)  
                parent2 = rouletteSelection(mattingPopulation) 
            elif (selectionTypeParent== 2):
                parent1 = tournamentSelection(mattingPopulation)  
                parent2 = tournamentSelection(mattingPopulation)
            child = crossover2(parent1, parent2)
            nextPopulation[i] = child
        nextPopulation = mutatePopulation(nextPopulation,
                                          elitismOffset)  # If ElitismOffset is 0 we will mutate all population
        return nextPopulation

    def crossover2(parent_1, parent_2):

        list_1 = []

        geneA = int(random.random() * len(parent_1))
        geneB = int(random.random() * len(parent_1))

        startGene = min(geneA, geneB)
        endGene = max(geneA, geneB)

        for i in range(startGene, endGene):
            list_1.append(parent_1[i])

        list_2 = [item for item in parent_2 if item not in list_1]
        list = []
        i_1 = 0
        i_2 = 0

        for i in range(len(Cities)):
            if (i in range(startGene, endGene)):
                list.append(list_1[i_1])
                i_1 += 1
            else:
                list.append(list_2[i_2])
                i_2 += 1

        return Route(list)

    def mutatePopulation(population, offset):  # New Version
        for i in range(offset, population.populationSize()):
            mutateRoute(population[i])
        return population

    def mutateRoute(route):  
        if random.random() < mutationRate:
            id_1 = random.randint(0, len(route) - 1)
            id_2 = random.randint(0, len(route) - 1)
            # SWAP
            tmpRoute = route[id_1]
            route.addCity(id_1, route[id_2])
            route.addCity(id_2, tmpRoute)

    def tournamentSelection(population):  
        tPopulation = Population(tournamentSelectionSize, False)
        for i in range(0, tournamentSelectionSize):
            id = random.randint(0, len(population) - 1)
            tPopulation[i] = population[id]
        return tPopulation.getFittest()

    def rouletteSelection(population):
        rouletteList = population.getRouletteList()
        maxvalue = rouletteList[population.populationSize() - 1]
        random_id = random.uniform(0.0, maxvalue)
        for i in range(0, len(rouletteList) - 1):
            if (rouletteList[i] <= random_id < rouletteList[i + 1]):
                return population[i]
        return population[len(rouletteList) - 1]

    def mattingCreatorPool(population):
        size = int(len(population) / 2)
        mattingPopulation = Population(size, False)
        for i in range(0, len(mattingPopulation)):
            if (selectionTypePool== 1) :
                mattingPopulation[i] = rouletteSelection(population)
            elif(selectionTypePool== 2) :
                mattingPopulation[i] = tournamentSelection(population)
        return mattingPopulation


    for id in range(0, len(Matrix)):
        city = City(id)
        Cities.append(city)

    t1 = time.time()
    # Initialize population
    pop = Population(PopulationSize, True)
    initial_distance = pop.getFittest().getDistance()

    # Evolve population for N generations
    pop = evolvePopulation(pop)
    
    for i in range(0, GenerationSize):
        # YOU CAN CHANGE THE MAXIMUM TIME HERE
        if timed:
            if time.time() - t1 > 300:
                return initial_distance, pop.getFittest().getDistance(), pop.getFittest(), time.time() - t1
        pop = evolvePopulation(pop)

    t2 = time.time()
    final_time = t2 - t1
    final_distance = pop.getFittest().getDistance()
    final_path = pop.getFittest()

    return initial_distance, final_distance, final_path, final_time

## Sample matrix


GA takes a long time to find a best distance when number of cities grows.<br><p>
In order to make experements with parameters and to discover which parameters are the best it was decided to use a <br>simpler 10x10 adjacency matrix, because with the grows of number of "cities" execution time becomes very high.<br>
<p>P.S: it is not correct to extrapolate the results of one datasets to all possible datasets, but because parameters tuning is quite a hard question for this type of algorithm (because of high dependencies between data and parameters), we'll still use best parameters from this solution for further evaluation, because the type of datasets in TSPLIB is simular.


In [ ]:
X = [[25,50],
     [15,40],
     [33,65],
     [24,19],
     [7,26],
     [0,37],
     [47,5],
     [56,23],
     [39,18],
     [14,31] ]
data = euclidean_distances(X, X)
data.tolist()

## First experiment.

Let set-up small mutation rate, elitism and roulette method for selecting matting pool and parents.

In [173]:
gas = []
res = []
time_res = []

for runs in range(10):
    gas.append(GA_approach(data, PopulationIntended=1000, GenerationIntended=25, mutationRateIntended=0.05, tournamentSelectionSizeIntended=125,
                selectionTypeParentIntended = 1, selectionTypePoolIntended = 1, elitismIntended=True, timed=False ))

for i in range(10) :
    print("Initial distance: " + str(gas[i][0])+ " Obtained distance: " + str(gas[i][1])+ "  Time: " + str(gas[i][3]))
print("")
res = [np.mean([idx[1] for idx in gas]), np.std([idx[1] for idx in gas])]
print("Obtained mean distance results: " + str(res[0]))
time_res = [np.mean([idx[3] for idx in gas]), np.std([idx[3] for idx in gas])]
print("Obtained mean time results: " + str(time_res[0]))



Initial distance: 220.70995808174214 Obtained distance: 182.00794886736682  Time: 21.675868034362793
Initial distance: 199.35266475257598 Obtained distance: 182.00794886736682  Time: 21.85685420036316
Initial distance: 208.26229421955466 Obtained distance: 182.0079488673668  Time: 24.276663064956665
Initial distance: 203.30901343604296 Obtained distance: 182.00794886736682  Time: 21.870076894760132
Initial distance: 216.61069083994772 Obtained distance: 182.0079488673668  Time: 21.603106021881104
Initial distance: 209.65794401484143 Obtained distance: 182.0079488673668  Time: 21.450886964797974
Initial distance: 220.52070438769508 Obtained distance: 182.00794886736682  Time: 21.494857788085938
Initial distance: 210.7886819400554 Obtained distance: 182.0079488673668  Time: 21.45267605781555
Initial distance: 222.04226564209054 Obtained distance: 182.00794886736682  Time: 21.417556285858154
Initial distance: 211.16872234593234 Obtained distance: 182.00794886736682  Time: 21.2677760124206

## Second experiment.

Let set-up hight mutation rate, elitism and roulette method for selecting matting pool and parents.

In [174]:
gas = []
res = []
time_res = []

for runs in range(10):
    gas.append(GA_approach(data, PopulationIntended=1000, GenerationIntended=25, mutationRateIntended=0.75, tournamentSelectionSizeIntended=125,
                selectionTypeParentIntended = 1, selectionTypePoolIntended = 1, elitismIntended=True, timed=False ))

for i in range(10) :
    print("Initial distance: " + str(gas[i][0])+ " Obtained distance: " + str(gas[i][1])+ "  Time: " + str(gas[i][3]))
print("")
res = [np.mean([idx[1] for idx in gas]), np.std([idx[1] for idx in gas])]
print("Obtained mean distance results: " + str(res[0]))
time_res = [np.mean([idx[3] for idx in gas]), np.std([idx[3] for idx in gas])]
print("Obtained mean time results: " + str(time_res[0]))



Initial distance: 205.64883009425162 Obtained distance: 182.00794886736682  Time: 21.867130756378174
Initial distance: 205.48032911883624 Obtained distance: 185.159773368448  Time: 21.799912929534912
Initial distance: 196.85340521243148 Obtained distance: 190.14577611291256  Time: 22.078543186187744
Initial distance: 214.66960783313613 Obtained distance: 185.15977336844801  Time: 21.496378183364868
Initial distance: 205.6467420198123 Obtained distance: 189.46462676787075  Time: 21.238338947296143
Initial distance: 214.33913002270316 Obtained distance: 190.14577611291256  Time: 22.449644804000854
Initial distance: 215.9624636831407 Obtained distance: 186.96536722772626  Time: 21.825464010238647
Initial distance: 210.5455244591053 Obtained distance: 185.159773368448  Time: 21.657427072525024
Initial distance: 208.05200637865977 Obtained distance: 186.96536722772626  Time: 22.144958972930908
Initial distance: 211.42132857573782 Obtained distance: 185.159773368448  Time: 21.588551998138428

## Third experiment.

Taking into account results obtained above, lets save the small mutation rate for all further experiments.
Let set-up no elitism for this time and save roulette method for selecting matting pool and parents.

In [176]:
gas = []
res = []
time_res = []

for runs in range(10):
    gas.append(GA_approach(data, PopulationIntended=1000, GenerationIntended=25, mutationRateIntended=0.05, tournamentSelectionSizeIntended=125,
                selectionTypeParentIntended = 1, selectionTypePoolIntended = 1, elitismIntended=False, timed=False ))

for i in range(10) :
    print("Initial distance: " + str(gas[i][0])+ " Obtained distance: " + str(gas[i][1])+ "  Time: " + str(gas[i][3]))
print("")
res = [np.mean([idx[1] for idx in gas]), np.std([idx[1] for idx in gas])]
print("Obtained mean distance results: " + str(res[0]))
time_res = [np.mean([idx[3] for idx in gas]), np.std([idx[3] for idx in gas])]
print("Obtained mean time results: " + str(time_res[0]))


Initial distance: 211.77168601944228 Obtained distance: 185.15977336844801  Time: 22.453171968460083
Initial distance: 213.7982114332758 Obtained distance: 195.85233553553905  Time: 21.53212022781372
Initial distance: 211.22199707527128 Obtained distance: 193.0376484738677  Time: 22.27268695831299
Initial distance: 220.64488273970787 Obtained distance: 196.85340521243145  Time: 21.711368083953857
Initial distance: 218.60851681351804 Obtained distance: 195.85233553553905  Time: 22.42076015472412
Initial distance: 217.7502552260255 Obtained distance: 202.93061027785032  Time: 23.44774603843689
Initial distance: 206.53509261598526 Obtained distance: 182.00794886736682  Time: 22.45192313194275
Initial distance: 206.88203514203997 Obtained distance: 205.4801807514585  Time: 21.433953046798706
Initial distance: 204.3130371440865 Obtained distance: 201.3418886203895  Time: 21.927569150924683
Initial distance: 204.93510914001226 Obtained distance: 186.96536722772626  Time: 21.64775013923645

O

## Fourth experiment.

Taking into account results of the first and third experiments, lets conclude that the best option <br> 
is to save small mutation rate and elitism constraint.
<p>
For the next experiment lets set-up tournament method for selecting matting pool and parents.

In [180]:
gas = []
res = []
time_res = []

for runs in range(10):
    gas.append(GA_approach(data, PopulationIntended=1000, GenerationIntended=25, mutationRateIntended=0.05, tournamentSelectionSizeIntended=125,
                selectionTypeParentIntended = 2, selectionTypePoolIntended = 2, elitismIntended=True, timed=False ))

for i in range(10) :
    print("Initial distance: " + str(gas[i][0])+ " Obtained distance: " + str(gas[i][1])+ "  Time: " + str(gas[i][3]))
print("")
res = [np.mean([idx[1] for idx in gas]), np.std([idx[1] for idx in gas])]
print("Obtained mean distance results: " + str(res[0]))
time_res = [np.mean([idx[3] for idx in gas]), np.std([idx[3] for idx in gas])]
print("Obtained mean time results: " + str(time_res[0]))

Initial distance: 189.46462676787075 Obtained distance: 185.159773368448  Time: 22.51604390144348
Initial distance: 210.57320081001006 Obtained distance: 182.00794886736682  Time: 23.28362274169922
Initial distance: 213.09496434981463 Obtained distance: 185.159773368448  Time: 23.12364387512207
Initial distance: 196.85340521243148 Obtained distance: 182.00794886736682  Time: 24.127399921417236
Initial distance: 212.09641677674014 Obtained distance: 182.0079488673668  Time: 26.53110933303833
Initial distance: 215.20061353915804 Obtained distance: 185.159773368448  Time: 24.421208143234253
Initial distance: 207.2859229781145 Obtained distance: 182.00794886736682  Time: 22.518490076065063
Initial distance: 200.06311094452425 Obtained distance: 182.0079488673668  Time: 24.11259889602661
Initial distance: 213.252434540867 Obtained distance: 182.00794886736682  Time: 23.96954917907715
Initial distance: 182.00794886736682 Obtained distance: 182.00794886736682  Time: 23.43302321434021

Obtaine

## Fifth experiment.

From the experiment №1 and №4, we can observe that results for using same method for both selection <br> 
of matting pool and parents give quite simular results (indeed, experiment №4 takes 2 sec longer time <br> 
and produce not as good results as first experiment).
<p>
This time we'll keep small mutation rate and elitism constraint, but for matting pool selection we'll assign roulette method, for parents selection - tournament method.

In [182]:
gas = []
res = []
time_res = []

for runs in range(10):
    gas.append(GA_approach(data, PopulationIntended=1000, GenerationIntended=25, mutationRateIntended=0.05, tournamentSelectionSizeIntended=125,
                selectionTypeParentIntended = 2, selectionTypePoolIntended = 1, elitismIntended=True, timed=False ))

for i in range(10) :
    print("Initial distance: " + str(gas[i][0])+ " Obtained distance: " + str(gas[i][1])+ "  Time: " + str(gas[i][3]))
print("")
res = [np.mean([idx[1] for idx in gas]), np.std([idx[1] for idx in gas])]
print("Obtained mean distance results: " + str(res[0]))
time_res = [np.mean([idx[3] for idx in gas]), np.std([idx[3] for idx in gas])]
print("Obtained mean time results: " + str(time_res[0]))

Initial distance: 215.11549882425166 Obtained distance: 182.0079488673668  Time: 25.95058298110962
Initial distance: 209.7887282976291 Obtained distance: 185.159773368448  Time: 25.46117901802063
Initial distance: 214.58558774158658 Obtained distance: 182.00794886736682  Time: 25.18635106086731
Initial distance: 208.6694628057879 Obtained distance: 182.00794886736682  Time: 25.47010111808777
Initial distance: 209.2197637258093 Obtained distance: 182.00794886736682  Time: 38.934325218200684
Initial distance: 189.46462676787073 Obtained distance: 182.0079488673668  Time: 35.68397617340088
Initial distance: 197.32433567776013 Obtained distance: 182.00794886736685  Time: 34.412516832351685
Initial distance: 199.4080478726948 Obtained distance: 182.00794886736682  Time: 26.612240314483643
Initial distance: 197.32433567776013 Obtained distance: 182.00794886736685  Time: 25.398775100708008
Initial distance: 216.53847763016884 Obtained distance: 182.00794886736682  Time: 25.11417293548584

Obt

## Sixth experiment.
To make the full observation, for this experiment 
we'll again keep small mutation rate and elitism constraint, but for matting pool selection we'll assign tournament method, for parents selection - roulette method.

In [183]:
gas = []
res = []
time_res = []

for runs in range(10):
    gas.append(GA_approach(data, PopulationIntended=1000, GenerationIntended=25, mutationRateIntended=0.05, tournamentSelectionSizeIntended=125,
                selectionTypeParentIntended = 1, selectionTypePoolIntended = 2, elitismIntended=True, timed=False ))

for i in range(10) :
    print("Initial distance: " + str(gas[i][0])+ " Obtained distance: " + str(gas[i][1])+ "  Time: " + str(gas[i][3]))
print("")
res = [np.mean([idx[1] for idx in gas]), np.std([idx[1] for idx in gas])]
print("Obtained mean distance results: " + str(res[0]))
time_res = [np.mean([idx[3] for idx in gas]), np.std([idx[3] for idx in gas])]
print("Obtained mean time results: " + str(time_res[0]))

Initial distance: 194.82507613761564 Obtained distance: 182.0079488673668  Time: 19.067359924316406
Initial distance: 207.88802863820976 Obtained distance: 182.00794886736685  Time: 19.25370192527771
Initial distance: 201.87165936994631 Obtained distance: 182.0079488673668  Time: 19.263166904449463
Initial distance: 220.97283945523438 Obtained distance: 182.0079488673668  Time: 19.465701818466187
Initial distance: 208.6694628057879 Obtained distance: 182.0079488673668  Time: 21.011018991470337
Initial distance: 216.8345749354819 Obtained distance: 182.00794886736682  Time: 19.559524059295654
Initial distance: 195.85233553553905 Obtained distance: 182.0079488673668  Time: 18.312291860580444
Initial distance: 213.81595418670386 Obtained distance: 182.0079488673668  Time: 18.28362202644348
Initial distance: 210.5732008100101 Obtained distance: 182.0079488673668  Time: 18.206670999526978
Initial distance: 185.159773368448 Obtained distance: 182.0079488673668  Time: 18.32015109062195

Obtai

In the last experiment both distance and time results outperform.
That is why after a series of 6 experiments we can conclude that the best parameters option for proposed datasets is:

    *small mutation rate
    *elitism
    *roulette selection for parents
    *tournament selection for mating pool
    
In the next series of experiments we'll try to find best combination between population, generation and tournament selection size.


## Seventh experiment.
The new goal of experiment is to find out the most appropriate combination of population, generation and tournament selection sizes.
Let set up the proportion between Population, Generation and Tournament as -> 100%, 20%, 40%

In [188]:
route = GA_approach(data, PopulationIntended=2500, GenerationIntended=500, mutationRateIntended=0.05, tournamentSelectionSizeIntended=1000,selectionTypeParentIntended = 1, selectionTypePoolIntended = 2, elitismIntended=True, timed=True)

print("Initial distance: " + str(route[0]) 
          + " Final_distance: " + str(route[1]) 
          + " Time: " + str(route[3]))

Initial distance: 205.48018075145853 Final_distance: 182.0079488673668 Time: 301.24504804611206


Experiment №7 shows that we've selected not correct population, cause it converge to optimal solution in a very long period of time.

## Eighth experiment.

For this experiment we'll decrease population, but increase generation number and tournament selection size.
Let set up the proportion between Population, Generation and Tournament as -> 20%, 100%, 17%

In [219]:
gas = []
res = []
time_res = []

for runs in range(20):
    gas.append(GA_approach(data, PopulationIntended=100, GenerationIntended=500, mutationRateIntended=0.05, tournamentSelectionSizeIntended=85,selectionTypeParentIntended = 1, selectionTypePoolIntended = 2, elitismIntended=True, timed=True))

for i in range(20) :
    print("Initial distance: " + str(gas[i][0])+ " Obtained distance: " + str(gas[i][1])+ "  Time: " + str(gas[i][3]))
print("")
res = [np.mean([idx[1] for idx in gas]), np.std([idx[1] for idx in gas])]
print("Obtained mean distance results: " + str(res[0]))
time_res = [np.mean([idx[3] for idx in gas]), np.std([idx[3] for idx in gas])]
print("Obtained mean time results: " + str(time_res[0]))


Initial distance: 233.9068421208144 Obtained distance: 182.0079488673668  Time: 9.785682201385498
Initial distance: 252.2363296398811 Obtained distance: 185.159773368448  Time: 9.662638187408447
Initial distance: 247.84053671019328 Obtained distance: 182.00794886736682  Time: 9.035399913787842
Initial distance: 222.09563299118622 Obtained distance: 185.159773368448  Time: 9.16040301322937
Initial distance: 247.8834688548872 Obtained distance: 182.00794886736685  Time: 9.02505612373352
Initial distance: 248.5234789178982 Obtained distance: 182.00794886736682  Time: 9.30122184753418
Initial distance: 236.1136283378824 Obtained distance: 185.159773368448  Time: 9.018712043762207
Initial distance: 254.06370306044954 Obtained distance: 182.00794886736682  Time: 9.099674701690674
Initial distance: 232.94333254593556 Obtained distance: 182.00794886736682  Time: 9.142814874649048
Initial distance: 248.64613639908458 Obtained distance: 185.159773368448  Time: 9.03393816947937
Initial distance: 

## Ninth experiment.

Let's try to optimize time needed for convergency to optimal solution. We'll keep Generation number, but we'll decrease Population and Tournament sizes, as follow: Population -> 10%, Generation -> 100%, Tournament ->7%

In [220]:
gas = []
res = []
time_res = []

for runs in range(20):
    gas.append(GA_approach(data, PopulationIntended=50, GenerationIntended=500, mutationRateIntended=0.05, tournamentSelectionSizeIntended=35,selectionTypeParentIntended = 1, selectionTypePoolIntended = 2, elitismIntended=True, timed=True))

for i in range(20) :
    print("Initial distance: " + str(gas[i][0])+ " Obtained distance: " + str(gas[i][1])+ "  Time: " + str(gas[i][3]))
print("")
res = [np.mean([idx[1] for idx in gas]), np.std([idx[1] for idx in gas])]
print("Obtained mean distance results: " + str(res[0]))
time_res = [np.mean([idx[3] for idx in gas]), np.std([idx[3] for idx in gas])]
print("Obtained mean time results: " + str(time_res[0]))


Initial distance: 239.91567288866753 Obtained distance: 185.15977336844801  Time: 2.4685819149017334
Initial distance: 255.00939770578924 Obtained distance: 209.88992092563305  Time: 2.3501288890838623
Initial distance: 227.7358787650593 Obtained distance: 185.159773368448  Time: 2.3471930027008057
Initial distance: 255.1305605031658 Obtained distance: 182.0079488673668  Time: 2.3375980854034424
Initial distance: 226.7485957452526 Obtained distance: 182.00794886736682  Time: 2.363618850708008
Initial distance: 240.5976101193845 Obtained distance: 185.159773368448  Time: 2.3789520263671875
Initial distance: 252.7127347467086 Obtained distance: 182.0079488673668  Time: 2.333536148071289
Initial distance: 238.27669934540364 Obtained distance: 185.15977336844801  Time: 2.3479690551757812
Initial distance: 238.10542142090677 Obtained distance: 182.00794886736682  Time: 2.3423397541046143
Initial distance: 228.78873833893277 Obtained distance: 182.0079488673668  Time: 2.3371307849884033
Init

So, the most appropriate solution will be selection of Population, Generation and Tournament as -> 20%, 100%, 17% approximately

In [ ]:
## The final step is to test algorithm with one of the TSPLIB datasets.
berlin52 dataset was selected because it has relatively small number of dots, so it would converge faster.

In [224]:
with open("../../data/berlin52_array.txt", 'r') as myfile:
    data_2=myfile.read().replace('\n', '')
data_2= eval(data_2)
graph = gc.fully_connected_graph_from_coordinate_list(data_2)
data_2= graph.weighted_adjacency_matrix
print(data_2)


[[  inf  649. 1047. ...  596. 1422. 1716.]
 [ 649.   inf  604. ...  463.  995. 1484.]
 [1047.  604.   inf ...  478.  397.  909.]
 ...
 [ 596.  463.  478. ...   inf  830. 1151.]
 [1422.  995.  397. ...  830.   inf  625.]
 [1716. 1484.  909. ... 1151.  625.   inf]]


In [230]:
route = GA_approach(data_2, PopulationIntended=400, GenerationIntended=2000, mutationRateIntended=0.05, tournamentSelectionSizeIntended=340,selectionTypeParentIntended = 1, selectionTypePoolIntended = 2, elitismIntended=True, timed=False)

print("Initial distance: " + str(route[0]) 
          + " Final_distance: " + str(route[1]) 
          + " Time: " + str(route[3]))

Initial distance: 24562.0 Final_distance: 11208.0 Time: 594.6323108673096


In 10 min the best solution found is 11208 km, comparing to optimum route for dataset - 7542. 
It means that more time (generations) is needed to achive convergency.


In [229]:
route = GA_approach(data_2, PopulationIntended=800, GenerationIntended=6000, mutationRateIntended=0.05, tournamentSelectionSizeIntended=700,selectionTypeParentIntended = 1, selectionTypePoolIntended = 2, elitismIntended=True, timed=False)

print("Initial distance: " + str(route[0]) 
          + " Final_distance: " + str(route[1]) 
          + " Time: " + str(route[3]))

Initial distance: 23121.0 Final_distance: 9575.0 Time: 6888.462126016617


In 100 min the best solution found is 9575 km, comparing to optimum route for dataset - 7542. 
